# Access to data

NablaDFT includes three types of databases:

1. **Energy database.** There are molecule structure, energy, and forces. Data are available via the atomic simulation environment (ASE) interface.
2. **Hamiltonian database.** There are molecule structure, energy, forces, hamiltonian and overlap matrix. Data are available via nabla2DFT custom access interface.
3. **Raw psi4 wave function.** There are serialized Psi4 wavefunction. Data are available via psi4 or numpy interfaces.

Each database has specific atom units, order of records, and order of atomic orbitals in Hamiltonians. In this tutorial, we show how to load and visualize some data. Advanced processing of metadata and Hamiltonians are described in the following lessons.

The smallest split (`train-tiny`,  51 Mb) of the energy database is available with `wget`:

In [3]:
!wget https://a002dlils-kadurin-nabladft.obs.ru-moscow-1.hc.sbercloud.ru/data/nablaDFTv2/energy_databases/train_2k_v2_formation_energy_w_forces.db

An atomic simulation environment (ASE) package is necessary for processing energy databases. It also helps to visualize molecules.

In [ ]:
import ase
from ase.db import connect
from ase.units import Bohr
from ase.visualize import view

In [7]:
with connect("train_2k_v2_formation_energy_w_forces.db") as train_db:
    atom_row = train_db.get(1)
    row = atom_row.toatoms()

Energy databases store atom positions in Angstrom.

In [8]:
row.numbers, row.positions

You can check the data using visualization. You can check the data using visualization. ASE takes physical units from meta information.

In [9]:
view(row, viewer="x3d")

# Hamiltonian database

The smallest split of the Hamiltonian database (`train-tiny`) requires downloading of 14 Gb.

In [18]:
!wget https://a002dlils-kadurin-nabladft.obs.ru-moscow-1.hc.sbercloud.ru/data/nablaDFTv2/hamiltonian_databases/train_2k.db

We provide a custom class to access the Hamiltonian databases.

In [30]:
from nablaDFT.dataset import HamiltonianDatabase

In [31]:
help(HamiltonianDatabase)

In [32]:
train = HamiltonianDatabase("train_2k.db")
# atoms numbers, atoms positions, energy, forces, core hamiltonian, overlap matrix, coefficients matrix
Z, R, E, F, H, S, C, moses_id, conformation_id = train[0]

In [33]:
Z, R

ASE cannot take physical units from metainformation. Angstrom to Bohr should be converted explicitly. You can check the correctness of units using ASE atomic visualization. If atomic units are correct, then each atom will drawn as a sphere. The spheres should touch each other but not intersect or stay aside.

In [15]:
atom = ase.Atoms(Z, R * Bohr)
view(atom, viewer="x3d")

# Raw psi4 wave function

You can upload PSI4 wavefunctions into the PSI4, or into numpy. Numpy-way is simpler because it is not required to install Psi4.

In [ ]:
!wget https://a002dlils-kadurin-nabladft.obs.ru-moscow-1.hc.sbercloud.ru/data/moses_wfns_big/wfns_moses_conformers_archive_0.tar
!tar -xf wfns_moses_conformers_archive_0.tar
!cd mnt/sdd/data/moses_wfns_big/
!ls mnt/sdd/data/moses_wfns_big/

In [34]:
import numpy as np

In [35]:
data = np.load("mnt/sdd/data/moses_wfns_big/wfn_conf_50000_0.npy", allow_pickle=True).tolist()

In [36]:
Z = data["molecule"]["elez"]
R = data["molecule"]["geom"].reshape((-1, 3))
Z, R

In [37]:
atom = ase.Atoms(Z, R)
view(atom, viewer="x3d")

In [38]:
wfn = np.load("mnt/sdd/data/moses_wfns_big/wfn_conf_50000_0.npy", allow_pickle=True).tolist()
orbital_matrix_a = wfn["matrix"]["Ca"]  # alpha orbital coefficients
orbital_matrix_b = wfn["matrix"]["Cb"]  # betta orbital coefficients
density_matrix_a = wfn["matrix"]["Da"]  # alpha electonic density
density_matrix_b = wfn["matrix"]["Db"]  # betta electonic density
aotoso_matrix = wfn["matrix"]["aotoso"]  # atomic orbital to symmetry orbital transformation matrix
core_hamiltonian_matrix = wfn["matrix"]["H"]  # core Hamiltonian matrix
fock_matrix_a = wfn["matrix"]["Fa"]  # DFT alpha Fock matrix
fock_matrix_b = wfn["matrix"]["Fb"]  # DFT betta Fock matrix

An advenced processing of wavefunctions and data is available from psi4 also. Note, that psi4 require compilation or conda install. More information about obtaining of PSI4 is available here https://psicode.org/psi4manual/master/build_obtaining.html

In [ ]:
import psi4

In [ ]:
wfn = psi4.core.Wavefunction.from_file("mnt/sdd/data/moses_wfns_big/wfn_conf_50000_0.npy")
psi4.oeprop(wfn, "MAYER_INDICES")
psi4.oeprop(wfn, "WIBERG_LOWDIN_INDICES")
psi4.oeprop(wfn, "MULLIKEN_CHARGES")
psi4.oeprop(wfn, "LOWDIN_CHARGES")
meyer_bos = wfn.array_variables()["MAYER INDICES"]  # Mayer bond indices
lodwin_bos = wfn.array_variables()["WIBERG LOWDIN INDICES"]  # Wiberg bond indices
mulliken_charges = wfn.array_variables()["MULLIKEN CHARGES"]  # Mulliken atomic charges
lowdin_charges = wfn.array_variables()["LOWDIN CHARGES"]  # Löwdin atomic charges